In [ ]:
# этот скрипт делает запрос к openai api.

In [8]:
#%pip install numpy
#%pip install pandas
#%pip install python-dotenv

In [35]:
import base64
import requests
from dotenv import load_dotenv
import os
import json

# Загрузка переменных из файла .env
load_dotenv()

# Получение значения переменной OPENAI_API_KEY
api_key = os.getenv('OPENAI_API_KEY')

headers = {
  "Content-Type": "application/json",
  "Authorization": f"Bearer {api_key}"
}

# Function to encode the image
def encode_image(image_path):
  with open(image_path, "rb") as image_file:
    return base64.b64encode(image_file.read()).decode('utf-8')

In [11]:
# Path to your image
image_path = "img/1T.png"

# Getting the base64 string
base64_image = encode_image(image_path)

In [19]:
payload = {
  "model": "gpt-4o",
  "messages": [
    {
      "role": "system", 
      "content": "You are a professional quantitative analyst specializing in financial market analysis and price" 
              + " forecasting. You always spend a few sentences explaining background context, assumptions, and" 
              + " step-by-step thinking BEFORE you try to answer a question to ensure accuracy. You conduct a"
              + " thorough analysis of the overall market situation before providing forecasts."
    },
    {
      "role": "user", 
      "content": [
        {
          "type": "text", 
          "text": "Could you please analyze the attached minute candlestick chart and provide forecasts for the next"
                + " 10 minutes. The main chart includes Bollinger Bands (period=20, color=black) and ATR"
                + " (period=10, color=red). The volume chart also includes RSI (period=14, color=black)."
                + " Below the volume chart is a panel with MACD (fast=12, slow=26, signal=9). Please include the"
                + " predicted closing price for the next 10 minutes as well as the probabilities of"
                + " price increase and decrease."
        },
        {
          "type": "image_url",
          "image_url": {
            "url": f"data:image/jpeg;base64,{base64_image}"
          },
        }
      ],
    }
  ],
  "max_tokens": 1000
}

response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload)
j = response.json()
j

{'id': 'chatcmpl-9VMY8bEWdOngtxUMS0GS2Bl9EA3mA',
 'object': 'chat.completion',
 'created': 1717261876,
 'model': 'gpt-4o-2024-05-13',
 'choices': [{'index': 0,
   'message': {'role': 'assistant',
    'content': '### Market Context\n\n1. **Bollinger Bands**:\n   - The price is primarily trading at or below the lower Bollinger Band, indicating a strong downward trend.\n   - The distance between the bands is relatively wide, indicating increased volatility.\n\n2. **Average True Range (ATR)**:\n   - ATR is increasing, signaling rising volatility and potential for larger price movements.\n\n3. **Relative Strength Index (RSI)**:\n   - RSI is below the 30 level, indicating that the market is currently oversold. This could suggest a potential for a short-term bounce or reversal.\n\n4. **Moving Average Convergence Divergence (MACD)**:\n   - The MACD line is substantially below the signal line and is also in negative territory, indicating strong bearish momentum.\n\n### Short-term Price Forecast

In [28]:
# Запустите следующий код в Jupyter Notebook, чтобы создать функцию, которая будет добавлять новую ячейку с содержимым:
from IPython.display import display, Markdown, Javascript

def create_markdown_cell(content):
    display(Markdown(content))

# Создайте функцию, которая удаляет следующую ячейку, если она имеет тип Markdown:
def delete_next_markdown_cell():
    display(Javascript("""
    var currentCell = Jupyter.notebook.get_selected_index();
    var nextCell = Jupyter.notebook.get_cell(currentCell + 1);
    if (nextCell && nextCell.cell_type === 'markdown') {
        Jupyter.notebook.delete_cell(currentCell + 1);
    }
    """))

# Извлечение значения поля content
content = j['choices'][0]['message']['content']

# Удаление следующей Markdown ячейки, если она существует
delete_next_markdown_cell()

# Создание новой markdown ячейки с содержимым
create_markdown_cell(content)

<IPython.core.display.Javascript object>

### Market Context

1. **Bollinger Bands**:
   - The price is primarily trading at or below the lower Bollinger Band, indicating a strong downward trend.
   - The distance between the bands is relatively wide, indicating increased volatility.

2. **Average True Range (ATR)**:
   - ATR is increasing, signaling rising volatility and potential for larger price movements.

3. **Relative Strength Index (RSI)**:
   - RSI is below the 30 level, indicating that the market is currently oversold. This could suggest a potential for a short-term bounce or reversal.

4. **Moving Average Convergence Divergence (MACD)**:
   - The MACD line is substantially below the signal line and is also in negative territory, indicating strong bearish momentum.

### Short-term Price Forecast

Given the current indicators, the market exhibits strong bearish signals. However, the oversold condition per RSI creates a slight counterpoint suggesting a possibility of a minor pullback or temporary reversal. 

### Predictions & Probabilities

- **Expected Closing Price for Next 10 Minutes**: The next few candlesticks are likely to maintain or slightly recover from their current downward trajectory:
  - **Predicted Closing Price**: Around 67580 to 67650.

- **Price Increase Probability**:
  - Due to the RSI being oversold, there is roughly a 30% chance of a minor upward correction.

- **Price Decrease Probability**:
  - Given the overall strong bearish sentiment indicated by the Bollinger Bands, ATR, and MACD, a continued decline in price has about a 70% probability.

### Summary

Expect the bearish trend to dominate in the immediate term with the possibility of minor recoveries. If volume and RSI begin to show signs of divergence, this could indicate a potential reversal. Monitor for any significant change in volume and RSI movement out of the oversold zone closely as they might signal a shift.

In [32]:
payload = {
  "model": "gpt-4o",
  "response_format": { "type": "json_object" },
  "messages": [
    {
      "role": "system", 
      "content": """You are an LLM assistant specialized in converting financial analysis text into structured JSON format.
 Your task is to extract key numerical values and probabilities from the provided analysis and format
 them into a JSON object with the following fields: 'h' for the predicted highest price, 'l' for the
 predicted lowest price, 'p_up' for the probability of price increase, 'p_down' for the probability
 of price decrease, and 'error' for any potential issues or discrepancies noted during extraction.
 Ensure the JSON output accurately reflects the information in the analysis, adheres to the following
 JSON Schema:

```
{
  "$schema": "http://json-schema.org/draft-07/schema#",
  "type": "object",
  "properties": {
    "h": {
      "type": "number",
      "description": "Predicted highest price"
    },
    "l": {
      "type": "number",
      "description": "Predicted lowest price"
    },
    "p_up": {
      "type": "number",
      "description": "Probability of price increase",
      "minimum": 0,
      "maximum": 100
    },
    "p_down": {
      "type": "number",
      "description": "Probability of price decrease",
      "minimum": 0,
      "maximum": 100
    },
    "error": {
      "type": "string",
      "description": "Error message if any issue is found"
    }
  },
  "additionalProperties": false
}
```

Respond only in JSON format without any additional text, as your response will be processed automatically by a program."""
    },
    {
      "role": "user", 
      "content": [
        {
          "type": "text", 
          "text": '### Market Context\n\n1. **Bollinger Bands**:\n   - The price is primarily trading at or below the lower Bollinger Band, indicating a strong downward trend.\n   - The distance between the bands is relatively wide, indicating increased volatility.\n\n2. **Average True Range (ATR)**:\n   - ATR is increasing, signaling rising volatility and potential for larger price movements.\n\n3. **Relative Strength Index (RSI)**:\n   - RSI is below the 30 level, indicating that the market is currently oversold. This could suggest a potential for a short-term bounce or reversal.\n\n4. **Moving Average Convergence Divergence (MACD)**:\n   - The MACD line is substantially below the signal line and is also in negative territory, indicating strong bearish momentum.\n\n### Short-term Price Forecast\n\nGiven the current indicators, the market exhibits strong bearish signals. However, the oversold condition per RSI creates a slight counterpoint suggesting a possibility of a minor pullback or temporary reversal. \n\n### Predictions & Probabilities\n\n- **Expected Closing Price for Next 10 Minutes**: The next few candlesticks are likely to maintain or slightly recover from their current downward trajectory:\n  - **Predicted Closing Price**: Around 67580 to 67650.\n\n- **Price Increase Probability**:\n  - Due to the RSI being oversold, there is roughly a 30% chance of a minor upward correction.\n\n- **Price Decrease Probability**:\n  - Given the overall strong bearish sentiment indicated by the Bollinger Bands, ATR, and MACD, a continued decline in price has about a 70% probability.\n\n### Summary\n\nExpect the bearish trend to dominate in the immediate term with the possibility of minor recoveries. If volume and RSI begin to show signs of divergence, this could indicate a potential reversal. Monitor for any significant change in volume and RSI movement out of the oversold zone closely as they might signal a shift.'
        }
      ],
    }
  ],
  "max_tokens": 1000
}

response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload)
j = response.json()
j

{'id': 'chatcmpl-9VQZErT3kZN1kycCPc5G1dPkvg1tV',
 'object': 'chat.completion',
 'created': 1717277320,
 'model': 'gpt-4o-2024-05-13',
 'choices': [{'index': 0,
   'message': {'role': 'assistant',
    'content': '{\n  "h": 67650,\n  "l": 67580,\n  "p_up": 30,\n  "p_down": 70,\n  "error": ""\n}'},
   'logprobs': None,
   'finish_reason': 'stop'}],
 'usage': {'prompt_tokens': 742, 'completion_tokens': 39, 'total_tokens': 781},
 'system_fingerprint': 'fp_319be4768e'}

In [38]:
json_string = j['choices'][0]['message']['content']

# Преобразование строки JSON в объект Python (словарь)
data = json.loads(json_string)

# Чтение значения 'h'
data["h"], data['l'], data['p_up'], data['p_down'], data['error']

(67650, 67580, 30, 70, '')